In [57]:
#create advanced DV reports
import astropy.io.fits as apf
import batman
import eleanor
import emcee
import getpass
import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import shutil
import time as tm
from astroquery.mast import Catalogs
import astropy.units as units
from astropy.wcs import WCS
import math
import astropy.io.fits as apf
from astropy.stats import sigma_clip
import warnings
warnings.filterwarnings("ignore")
from wotan import flatten
import lightkurve as lk
import corner
import numpy as np
import juliet
import matplotlib.gridspec as gridspec

from astropy.timeseries import BoxLeastSquares
from ldtk import LDPSetCreator, BoxcarFilter, TabulatedFilter, SVOFilter
from ldtk.filters import tess, sdss_z
from astroquery import svo_fps

import mr_forecast as mr


In [6]:
mdwarfs = pd.read_csv('Mdwarf_stars_T_logg_z.csv', skiprows = 52)

In [20]:
mdwarfs = mdwarfs.drop_duplicates(subset=['hostname']).reset_index(drop=True)

In [33]:
# tess_filter = SVOFilter('TESS/TESS.Red')

# ldtk_q1 = []
# ldtk_q2 = []
# for name in list(mdwarfs.hostname):
#     new_df = mdwarfs[mdwarfs.hostname == name]
#     print(new_df.st_teff[0])
#     teff = float(new_df.st_teff[0])
#     mean_teff_unc = (np.abs(new_df.st_tefferr1[0])+np.abs(new_df.st_tefferr2[0]))/2
#     logg = new_df.st_logg[0]
#     mean_logg_unc = (np.abs(new_df.st_loggerr1[0])+np.abs(new_df.st_loggerr2[0]))/2
#     z = new_df.st_met[0]
#     mean_z_unc = (np.abs(new_df.st_meterr1[0])+np.abs(new_df.st_meterr2[0]))/2
    
#     ###THIS NEXT FUNCTION IS GOING TO DOWNLOAD A LOT OF STUFF, SO MAKE SURE ITS WHAT YOU WANT TO DO BEFORE YOU RUN IT
#     sc = LDPSetCreator(teff=(teff, mean_teff_unc), 
#                        logg = (logg, mean_logg_unc),
#                        z=(z, mean_z_unc), 
#                        filters = [tess_filter], dataset = 'vis-lowres')

#     ps =sc.create_profiles(nsamples = 2000)
#     ps.resample_linear_z(300)
#     qc_tess, qe = ps.coeffs_qd()
#     print(qc_tess[0][0], qc_tess[0][1])
#     ldtk_q1.append(qc_tess[0][0])
#     ldtk_q2.append(qc_tess[0][1])

In [69]:
def get_oneSigmaNoise(integration, Tmag):
    return (np.exp(get_oneHourNoiseLnsigma(Tmag)) /
            np.sqrt(integration / 60.))


def get_oneHourNoiseLnsigma(Tmag):
    """
    TESS photometric error estimate [ppm] based on
    magnitude and Eq. on bottom of P24 of
    arxiv.org/pdf/1706.00495.pdf
    """
    F = 4.73508403525e-5
    E = -0.0022308015894
    D = 0.0395908321369
    C = -0.285041632435
    B = 0.850021465753
    lnA = 3.29685004771

    return (lnA + B * Tmag + C * Tmag**2 + D * Tmag**3 +
            E * Tmag**4 + F * Tmag**5)

max_Tmag = 18
min_Tmag = 6.5

lst_Tmag = np.arange(min_Tmag, max_Tmag, 0.01)


cdpp_model = []
for T in lst_Tmag:
    cdpp_model.append(get_oneSigmaNoise(60, T))
    
    
    
def calc_tdur(delta, sigma_1hr, SNR, Ntran):

    Tdur = ((SNR*sigma_1hr/delta)**2)/Ntran
    return Tdur

def calc_snr(rp, Rs, Tdur, sigma_1hr, Ntran=1):
    A = Ntran/(Rs**2)/sigma_1hr
    SNR = A*(rp**2)*(Tdur**0.5)
    return SNR


In [143]:
e = 0
b = 0
omega = 1
SunM  = 333000
SunR  = 109.076
# periods = 
durations = np.linspace(60, 12*60, 100)
radii = np.linspace(0.4, 20., 100)


# masses = []
# for i in radii:
#     Mmedian, Mplus, Mminus  = mr.Rstat2M(mean = i, std = 0.1, sample_size = 100)
#     masses.append(Mmedian)
# masses

TypeError: Field elements must be 2- or 3-tuples, got '0'

In [144]:
def calc_snr_grid(Mstar, Rstar, Tmag, duration_grid = durations, planet_size = radii):
    '''Inputs: 
    Mstar: host star mass in solar masses
    Rstar: host star radius in solar radii
    Tmag: TESS magnitude of host star
    duration_grid: list of possible durations to iterate over
    planet_size: sizes of planets to measure snr for
    Outputs:
    snr_array: array of snr values as a function of planet radius (x-axis) and tranist duration (y-axis)'''
    
    
    star_m = Mstar*SunM #converting to earth units
    star_r = Rstar*SunR #converting to earth units
    G = 6.67e-11 #Gravitational constant (m**3. * kg**-1 * s**-2)
    snr_array = np.zeros((len(duration_grid), len(planet_size)))
#     for ppp in period_grid:
        

#         a = (G*(star_m+masses_904[i])*5.972e+24*((ppp*24.*3600.)**2.)/(4.*math.pi**2.))**(1./3.)
#         a = a*215.*109.076/1.496e11

#         Tdur =ppp*((1./math.pi)*(TOI904MR/a)*np.sqrt(1-b**2.)*np.sqrt(1-e**2.)/(1. + e*np.sin(omega))*24)
    for i in range(len(duration_grid)):
        for j in range(len(planet_size)):
            SNR = calc_snr(planet_size[j], star_r, duration_grid[i], Ntran = 1, sigma_1hr =get_oneSigmaNoise(60, Tmag)*1E-6)
            snr_array[i][j] = SNR
    return snr_array

def snr_limit_grid(dur_rad_snr_array, snr_lower_limit, radii=radii, durations=durations):
    '''Input: 
        snr_array: array of snr values as a function of planet radius (x-axis) and tranist duration (y-axis)
        snr_lower_limit: the lowest possible snr to test
        Output: panda dataframe with radii and durations is the columns in indeces, only including snrs>min values'''
    dur_rad_snr_array[dur_rad_snr_array<snr_lower_limit] = np.nan
    df = pd.DataFrame(dur_rad_snr_array, columns = np.array(radii).astype(str), index = np.array(durations).astype(str))
    return df

def get_tuples_of_acceptable_dur_and_rp(snr_df):
    """create a list of tuples of durations and planet radii that create an acceptable snr"""
    tuples_lst = []
    for column in snr_df.columns:
        indx_list = snr_df.index[snr_df[column]>0].tolist()
        if len(indx_list)>0:
            tuples_lst.extend([(float(i), float(column)) for i in indx_list])
    min_dur = min([mmm[0]] for mmm in tuples_lst)
    min_rp  = min([mmm[1]] for mmm in tuples_lst)
    return tuples_lst, min_dur, min_rp

In [145]:
snr_limit_df = snr_limit_grid(calc_snr_grid(0.3, 0.3, 12.), 7.3)
tuples, min_dur, min_rp= get_tuples_of_acceptable_dur_and_rp(snr_limit_df)


In [146]:
tuples

[(173.33333333333334, 0.5979797979797981),
 (180.0, 0.5979797979797981),
 (186.66666666666669, 0.5979797979797981),
 (193.33333333333334, 0.5979797979797981),
 (200.0, 0.5979797979797981),
 (206.66666666666669, 0.5979797979797981),
 (213.33333333333334, 0.5979797979797981),
 (220.0, 0.5979797979797981),
 (226.66666666666669, 0.5979797979797981),
 (233.33333333333334, 0.5979797979797981),
 (240.0, 0.5979797979797981),
 (246.66666666666669, 0.5979797979797981),
 (253.33333333333334, 0.5979797979797981),
 (260.0, 0.5979797979797981),
 (266.6666666666667, 0.5979797979797981),
 (273.33333333333337, 0.5979797979797981),
 (280.0, 0.5979797979797981),
 (286.6666666666667, 0.5979797979797981),
 (293.33333333333337, 0.5979797979797981),
 (300.0, 0.5979797979797981),
 (306.6666666666667, 0.5979797979797981),
 (313.33333333333337, 0.5979797979797981),
 (320.0, 0.5979797979797981),
 (326.6666666666667, 0.5979797979797981),
 (333.33333333333337, 0.5979797979797981),
 (340.0, 0.5979797979797981),
 (3

In [147]:
min_dur, min_rp

([60.0], [0.5979797979797981])